<a href="https://colab.research.google.com/github/van-dang/DMRI-FEM-Cloud/blob/master/ExplicitImplementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About the code

In [0]:
# This program solves the Bloch-Torrey equation applied to computational diffusion MRI using 
# the finite element method coupled with the theta-method for the spatial discretization.

# The scope of usage: 
# (1) Single domains, Multilayered structures, manifolds
# (2) Membrane permeability for internal interfaces
#     Artificial permeability at the external interfaces
# (3) pure homogeneous Neumann BCs, (4) pseudo-periodic BCs

# Copyright (C) 2019 Van-Dang Nguyen (vdnguyen@kth.se)

# This file is part of DOLFIN.

# DOLFIN is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# DOLFIN is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU Lesser General Public License for more details.

# You should have received a copy of the GNU Lesser General Public License
# along with DOLFIN. If not, see <http://www.gnu.org/licenses/>.

# First added:  2017-10-10
# Last changed: 2019-04-25

# This demo is maintained by Van-Dang Nguyen
# Please report possible problems to vdnguyen@kth.se

# Setting a working environment with FEniCS

In [10]:
try:
      from google.colab import files
except:
      print("Running with local runtime")
import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()
    
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    from dolfin import *; from mshr import *

gmsh_dir=!which gmsh
if len(gmsh_dir)==0:
  !sudo apt-get -y update
  !sudo apt-get install -y gmsh unzip
    
import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot 
import time

import os, sys, shutil

import sympy as sp


clear_output()

dolfin_version = dolfin.__version__
print ('dolfin version:', dolfin_version)

# Disable warnings
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger("rothemain.rothe_utils")
logging.getLogger('UFL').setLevel(logging.WARNING)
logging.getLogger('FFC').setLevel(logging.WARNING)

import warnings
warnings.filterwarnings("ignore")


dolfin version: 2019.1.0


#Load pre-defined functions

In [0]:
import sympy as sp  
exists = os.path.isfile('DmriFemLib.py')
isupdate = False
if (exists==False or isupdate==True):
    if isupdate==True:
        !rm DmriFemLib.py
    print("Load pre-defined functions from GitHub")
    !wget --quiet https://raw.githubusercontent.com/van-dang/DMRI-FEM-Cloud/master/DmriFemLib.py
from DmriFemLib import *

# Working on the mesh

In [12]:
geo_choice = 1
if geo_choice == 1:
    mesh_file = "fru_M_100383_1D.xml"
else:    
    mesh_file = "fru_M_100383_3D.xml"
mesh_file_exists = os.path.isfile(mesh_file)
if (mesh_file_exists==False):
    !wget https://github.com/van-dang/DMRI-FEM-Cloud/raw/mesh/$mesh_file''.zip  
    !unzip -q $mesh_file''.zip
mymesh = Mesh(mesh_file);

--2019-07-23 16:01:05--  https://github.com/van-dang/DMRI-FEM-Cloud/raw/mesh/fru_M_100383_1D.xml.zip
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/van-dang/DMRI-FEM-Cloud/mesh/fru_M_100383_1D.xml.zip [following]
--2019-07-23 16:01:05--  https://raw.githubusercontent.com/van-dang/DMRI-FEM-Cloud/mesh/fru_M_100383_1D.xml.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.84.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.84.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33864 (33K) [application/zip]
Saving to: ‘fru_M_100383_1D.xml.zip.1’

fru_M_100383_1D.xml 100%[===================>]  33.07K  --.-KB/s    in 0.005s  

2019-07-23 16:01:05 (6.89 MB/s) - ‘fru_M_100383_1D.xml.zip.1’ saved [33864/33864]



# Solve the Bloch-Torrey equation

In [0]:
mri_simu = MRI_simulation()
mri_para = MRI_parameters()

bvalues = [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000,10000]
signal_array=[];
voi_array=[];
whole_vol_array = [];
for bvalue in bvalues:
    #################################################################################
    #########################  Pre-defined parameters ###############################
    mri_para.stype='PGSE'                                # sequence type
    mri_para.bvalue = bvalue;                              # bvalue
    mri_para.delta, mri_para.Delta = 10600, 43100        # time sequence
    mri_para.set_gradient_dir(mymesh, 1, 0, 0)           # gradient direction

    mri_para.T = mri_para.Delta+mri_para.delta
    mri_para.fs_sym = sp.Piecewise(
                    (  1., mri_para.s < mri_para.delta ),
                    (  0., mri_para.s < mri_para.Delta ),
                    ( -1., mri_para.s < mri_para.T ),
                    (  0., True )  
                ) 


    mri_para.Apply()
    mri_simu.k = 200;                                    # time-step size
    mri_simu.nskip = 100;                                  # frequency to print ouputs
    mydomain = MyDomain(mymesh, mri_para)
    mydomain.phase = None
    mydomain.PeriodicDir = [0, 0, 0];             # Direction of the periodicity
    mydomain.IsDomainPeriodic = False             # Confirm if the mesh if periodic
    mydomain.IsDomainMultiple = False             # Confirm if the mesh is multiple
    ################################################################################
    mydomain.Apply()   # Call Apply before setting the diffusion tensor
    ################################################################################
    # Impose the diffusion coefficient
    mydomain.D0 = 3e-3
    mydomain.D = mydomain.D0;
    #################################################################################

    # linsolver = PETScLUSolver("mumps")
    linsolver = KrylovSolver("bicgstab")

    # mri_simu.solve(mydomain, mri_para, linsolver)

    mri_simu.Dirac_Delta, mri_simu.u_0 = mri_simu.InitialCondition(mydomain, None)

    stepcounter = 0;

    M = MassMatrix(mydomain);

    mri_simu.t = 0;
    ft_prev  =  mri_para.time_profile(mri_simu.t);
    ift_prev = mri_para.itime_profile(mri_simu.t);

    start_time = time.time()

    while mri_simu.t < mri_para.T + mri_simu.k: # Time-stepping loop
        if stepcounter % mri_simu.nskip == 0:
            print('t: %6.2f '%mri_simu.t, 'T: %6.2f'%mri_para.T, 'dt: %.1f'%mri_simu.k,'qvalue: %e'%mri_para.qvalue,'Completed %3.2f%%'%(float(mri_simu.t)/float(mri_para.T+mri_simu.k)*100.0));

        ft = mri_para.time_profile(mri_simu.t);
        ift = mri_para.itime_profile(mri_simu.t);

        L = ThetaMethodL(ft_prev, ift_prev, mri_para, mri_simu, mydomain);
        A = 1/mri_simu.k*M + assemble(ThetaMethodF(ft, ift, mri_para, mri_simu, mydomain))

        b = assemble(L);

        linsolver.solve(A, mri_simu.u_0.vector(),b);

        ft_prev  = ft;
        ift_prev = ift;

        mri_simu.t += mri_simu.k;
        stepcounter += 1;

    mri_simu.elapsed_time = time.time() - start_time
    print("Successfully Completed! Elapsed time: %f seconds"%mri_simu.elapsed_time)   
    
    # Post-processing
    one = Function(mydomain.V)
    one.vector()[:] = 1
    whole_vol = assemble(one*dx)
    voi = assemble(mri_simu.Dirac_Delta*dx)
    if mydomain.IsDomainMultiple == True:
        u0r_0, u0i_0, u1r_0, u1i_0 = split(mri_simu.u_0)
        initial0 = assemble((1-mydomain.phase)*mri_simu.Dirac_Delta*dx);
        signal0 = assemble(((1-mydomain.phase)*u0r_0)*dx);
        initial1 = assemble(mydomain.phase*mri_simu.Dirac_Delta*dx)
        signal1 = assemble((mydomain.phase*u1r_0)*dx);
        signal = assemble((mydomain.phase*u1r_0+(1-mydomain.phase)*u0r_0)*dx);
        print('Signal on each compartment')
        print('Sum initial0: %.3e, Signal0: %.3e'%(initial0, signal0))
        print('Sum initial1: %.3e, Signal1: %.3e'%(initial1, signal1))
        out_text = 'b: %.3f, g: %.3f, q: %.3e, Signal: %.3e, Normalized signal: %.3e, kappa: %.3e, dt: %.3f, hmin: %.3f, whole_vol: %.3f, vol_of_interest: %.3f, Free signal: %.3e, elasped time %.3f (s)\n'%(mri_para.bvalue, mri_para.gvalue, mri_para.qvalue, signal, signal/voi, mydomain.kappa, mri_simu.k, mydomain.hmin, whole_vol, voi, exp(-mri_para.bvalue*mydomain.D0), mri_simu.elapsed_time)
        print(out_text)
        V0 = FunctionSpace(mydomain.mesh0, mydomain.Ve);
        V1 = FunctionSpace(mydomain.mesh1, mydomain.Ve);
        u0r_0p = project(u0r_0,V0)
        u1r_0p = project(u1r_0,V1)
        if mydomain.tdim==mydomain.gdim and not(plt==None):
            plt.figure(10000);
            plot(u0r_0p, cmap="coolwarm")
            plt.figure(10001);            
            plot(u1r_0p, cmap="coolwarm")  
        File("u0r.pvd")<<u0r_0p
        File("u1r.pvd")<<u1r_0p
    else:
        ur, ui = split(mri_simu.u_0)
        signal = assemble(ur*dx);
        out_text = 'b: %.3f, g: %.3f, q: %.3e, Signal: %.3e, Normalized signal: %.3e, dt: %.3f, hmin: %.3e, whole_vol: %.3f, vol_of_interest: %.3f, Free signal: %.3e, elasped time %.3f (s)\n'%(mri_para.bvalue, mri_para.gvalue, mri_para.qvalue, signal, signal/voi, mri_simu.k, mydomain.hmin, whole_vol, voi, exp(-mri_para.bvalue*mydomain.D0), mri_simu.elapsed_time)
        print(out_text)
        V = FunctionSpace(mydomain.mymesh,mydomain.Ve);
        ur_p = project(ur,V)
        if mydomain.tdim==mydomain.gdim and not(plt==None): 
            plt.figure(10000);
            plot(ur_p, cmap="coolwarm")
        File("ur.pvd")<<ur_p
    signal_array.append(signal)
    voi_array.append(voi)
    whole_vol_array.append(whole_vol);
    
    comm = MPI.comm_world
    rank = comm.Get_rank()    
    
    print("save to log.txt")
    outfile = open('log.txt', 'a')
    if int(rank) == 0:
            outfile.write(out_text)
    outfile.close()  

Domain size: xmin=-168.304857, ymin=-6.314897, zmin=-37.306071, xmax=90.533933, ymax=162.324000, zmax=38.639986
Function Space for Single Domains has 2 components
(ur, ui): r-real, i-imaginary
Initialize a standard function space.
t:   0.00  T: 53700.00 dt: 200.0 qvalue: 0.000000e+00 Completed 0.00%
t: 20000.00  T: 53700.00 dt: 200.0 qvalue: 0.000000e+00 Completed 37.11%
t: 40000.00  T: 53700.00 dt: 200.0 qvalue: 0.000000e+00 Completed 74.21%
Successfully Completed! Elapsed time: 4.039970 seconds
b: 0.000, g: 0.000, q: 0.000e+00, Signal: 2.462e+03, Normalized signal: 1.000e+00, dt: 200.000, hmin: 7.022e-01, whole_vol: 2462.124, vol_of_interest: 2462.124, Free signal: 1.000e+00, elasped time 4.040 (s)

save to log.txt
Domain size: xmin=-168.304857, ymin=-6.314897, zmin=-37.306071, xmax=90.533933, ymax=162.324000, zmax=38.639986
Function Space for Single Domains has 2 components
(ur, ui): r-real, i-imaginary
Initialize a standard function space.
t:   0.00  T: 53700.00 dt: 200.0 qvalue: 1

In [0]:
import numpy as np
plt.plot(bvalues, np.divide(signal_array,signal_array[0]))
plt.xlabel('b (s/mm^2)')
plt.ylabel('S(b)/S(0)')